In [1]:
# 인과적 언어 모델 - 생성 태스크
# 허깅페이스에서는 이전 토큰에 의해 다음 토큰이 결정된다는 의미에서 인과적 생성 Causal generation이라고 부름
# 국내에서는 대부분 생성 모델 generation model 이라고 지칭함

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    bos_token='</s>',  #kogpt2의 경우 특수토큰이 지정되지 않았으므로 지정해줌
    eos_token='</s>',  #지정 여부는 단어사전을 출력하거나 tokenizer.json 파일을 열어서 확인 가능
    unk_token='<unk>',
    pad_token='<pad>',
    mask_token='<mask>'
)
model = AutoModelForCausalLM.from_pretrained(model_name)

model #임베딩 레이어와 out_features 크기가 같음 - 입출력에서 동일한 단어사전을 공유해야 하기 때문임

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

2025-06-18 05:40:59.478371: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-18 05:40:59.485227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750225259.493387   42376 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750225259.495824   42376 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750225259.502081   42376 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [5]:
# 한국어 위키 데이터셋 사용 - 크기가 너무 크므로 일부만 사용한다
from datasets import load_dataset

split_dict = {
    "train": "train[:8000]",
    "test": "train[8000:10000]",
    "unused": "train[10000:]"
}
dataset = load_dataset("heegyu/kowikitext", split=split_dict)
del dataset["unused"]

dataset

README.md:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

kowikitext.py:   0%|          | 0.00/5.70k [00:00<?, ?B/s]

The repository for heegyu/kowikitext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/heegyu/kowikitext.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


kowikitext-20221001.parquet:   0%|          | 0.00/498M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'revid', 'url', 'title', 'text'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['id', 'revid', 'url', 'title', 'text'],
        num_rows: 2000
    })
})

In [6]:
# 모델이 이해할 수 있도록 자연어 데이터 인코딩 - 타이틀과 내용을 개행문자로 구분하여 합친 후 토큰화
tokenized_dataset = dataset.map(
    lambda batch: tokenizer([f"{ti}\n{te}" for ti,te in zip(batch["title"],batch["text"])]),
    batched=True,
    num_proc=2,
    remove_columns=dataset['train'].column_names
)

tokenized_dataset

Map (num_proc=2):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [31]:
# 전처리 수행 - 원본문서 끝에 <eos>를 넣고 최대 길이 512를 기준으로 그룹화
max_length = 512
def group_texts(batched_samples):
    sample = {k: v[0] for k,v in batched_samples.items()}

    if sample['input_ids'][-1] != tokenizer.eos_token_id:
        for k in sample.keys():
            sample[k].append(tokenizer.eos_token_id if k=="input_ids" else sample[k][-1])

    result = {k: [v[i: i+max_length] for i in range(0, len(v), max_length)] for k,v in sample.items()}

    return result



grouped_dataset = tokenized_dataset.map(
    group_texts,
    batched=True,
    batch_size=1,  # 하나의 샘플을 받아 여러 샘플을 출력하기 위한 설정
    num_proc=2
)

print(len(grouped_dataset['train'][0]['input_ids']))
print(grouped_dataset)

512
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 18365
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2400
    })
})


In [33]:
# 데이터셋에 없는 labels 칼럼 추가
from transformers import DataCollatorForLanguageModeling

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=False)
sample = collator([grouped_dataset['train'][i] for i in range(1)])

In [41]:
# 문장 생성
inputs = tokenizer("지난해 7월 ", return_tensors='pt').to(model.device)

outputs = model.generate(inputs.input_ids, max_new_tokens=100)
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(result[0])

지난해 7월 롯데백화점 본점 지하 1층 식품관에서 열린 ‘2018 롯데백화점 식품관’ 행사에서 선보인 ‘롯데백화점 식품관’ 브랜드는 총 4종이다.
이번 행사에서 선보인 ‘롯데백화점 식품관’은 롯데백화점 식품관 내 식품관 중 가장 큰 규모다.
롯데백화점 식품관은 식품관 내 식품관 내 식품관 중 가장 큰 규모다.
롯데백화점 식품관은 식품관 내 식품관 내 식품


In [47]:
# 생성 작업을 하나하나 구현하는 경우 - 복잡하고 속도도 느린 편임
import torch

input_ids = tokenizer("지난해 7월 ", return_tensors='pt').to(model.device).input_ids

with torch.no_grad():
    for _ in range(100):
        next_token = model(input_ids).logits[0, -1:].argmax(-1)
        input_ids = torch.cat((input_ids[0], next_token), -1).unsqueeze(0)

print(tokenizer.decode(input_ids[0].tolist()))

지난해 7월 롯데백화점 본점 지하 1층 식품관에서 열린 ‘2018 롯데백화점 식품관’ 행사에서 선보인 ‘롯데백화점 식품관’ 브랜드는 총 4종이다.
이번 행사에서 선보인 ‘롯데백화점 식품관’은 롯데백화점 식품관 내 식품관 중 가장 큰 규모다.
롯데백화점 식품관은 식품관 내 식품관 내 식품관 중 가장 큰 규모다.
롯데백화점 식품관은 식품관 내 식품관 내 식품
